# batch script for reproducibility

jupyter nbconvert --to script generate_results.ipynb

In [1]:
import matplotlib
matplotlib.use('Agg')
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap as lsc
from matplotlib import pyplot as plt

In [2]:
import pandas as pd
import numpy as np
import scipy
import sklearn 
import xgboost

import math
import random 
import re
import itertools

from collections import Counter 

from scipy.stats import binom_test
from scipy.stats import norm

from functools import partial

from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.preprocessing import PolynomialFeatures as plf
from sklearn import preprocessing

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.linear_model import Lasso 
from sklearn.linear_model import Ridge 
from xgboost import XGBRegressor

from ksuzuki_pylib import *

In [3]:
print('sklearn:', sklearn.__version__)
print('xgboost:', xgboost.__version__)
print('pandas:', pd.__version__)
print('numpy:', np.__version__)
print('scipy:', scipy.__version__)
print('matplotlib:', matplotlib.__version__)

sklearn: 0.21.0
xgboost: 0.82
pandas: 0.24.2
numpy: 1.16.3
scipy: 1.2.1
matplotlib: 3.0.3


In [4]:
np.random.seed(0)
random.seed(0)

In [5]:
# xgb ranges
range_depth = [6, 7, 8]
range_subsample = [0.8, 0.9, 1]
range_colsample = [0.8, 0.9, 1]
range_lr = [0.1, 0.05]

In [6]:
# ranges
logparams1 = [1e-2, 1e-1, 1e0, 1e1, 1e2]
logparams2 = [1.0, 1e1, 1e2, 1e3, 1e4, 1e5]
logparams3 = [1.0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
logparams4 = [1.0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]

range_lasso = logparams1
range_ridge = logparams1
range_krr_alpha = logparams3
range_krr_gamma = logparams3
range_svr_C = logparams2
range_svr_gamma = logparams4
range_svr_epsilon = logparams1

In [7]:
# for debugging
debug_run = False
#debug_run = True
if debug_run:
    # xgb ranges
    range_depth = [3,4]
    range_subsample = [0.9, 1]
    range_colsample = [0.9, 1]
    range_lr = [0.1, 0.05]
    
    # ranges
    logparams1 = [1e-2, 1e-1]
    logparams2 = [1.0, 1e1]
    logparams3 = [1.0, 1e-1]
    logparams4 = [1.0, 1e-1]
    
    range_lasso = logparams1
    range_ridge = logparams1
    range_krr_alpha = logparams3
    range_krr_gamma = logparams3
    range_svr_C = logparams2
    range_svr_gamma = logparams4
    range_svr_epsilon = logparams1

In [8]:
# OCM
ntree = 500
if debug_run:
    ntree = 10
    
# Lasso
cv_lasso = GridSearchCV(
    Lasso(random_state=929), cv=5, 
    param_grid={"alpha": range_lasso})

# Ridge
cv_ridge = GridSearchCV(
    Ridge(random_state=929), cv=5,
    param_grid={"alpha": range_ridge})

# KRR
cv_krr = GridSearchCV(
    KernelRidge(kernel='rbf'), cv=5,
    param_grid={"alpha": range_krr_alpha, 
                "gamma": range_krr_gamma})

# SVR
cv_svr = GridSearchCV(
    SVR(kernel='rbf'), cv=5,
    param_grid={"C": range_svr_C, 
                "gamma": range_svr_gamma, 
                "epsilon": range_svr_epsilon})

# XGB
cv_xgb = GridSearchCV(
    XGBRegressor(random_state=929,
                 n_estimators=ntree), cv=5,
    param_grid={"max_depth": range_depth, 
                "subsample": range_subsample, 
                "colsample_bytree": range_colsample,
                "learning_rate": range_lr})

## OCM data

In [9]:
ocm = pd.read_csv("input/OCM_matrix.csv").drop(['Unnamed: 0'], axis=1)

ocm_desc = pd.read_csv("input/OCM_matrix_desc.csv")
ind = ocm_desc['Unnamed: 0']
ocm_desc.index = ind
ocm_desc = ocm_desc.drop(['Unnamed: 0'], axis=1)

#exclude cat which has 'Th' in its compser
ind = ocm['Th'] == 0
ocm = ocm[ind]

#exclude data which has over 5 metals in his component
ind = (ocm.loc[:,:'Zr']>0).sum(axis=1)
ind = ind < 5
ocm = ocm.loc[ind]

ind = (ocm_desc.loc[:,:'Zr']>0).sum(axis=1)
ind = ind < 5
ocm_desc = ocm_desc.loc[ind]

ocm = ocm.drop(['Support_Co'],axis=1)
ocm_desc = ocm_desc.drop(['Support_Co'],axis=1)

# rename columns
sup = ocm.loc[:,'Support_Zr':'Support_Si'].columns
prom = ocm.loc[:,'Promotor_B':'Promotor_S'].columns
comp = ocm.loc[:,:'Zr'].columns
cond = ocm.loc[:,'Temperature, K':'P total, bar'].columns
prep = ocm.loc[:,'Impregnation':'Therm.decomp.'].columns

#exclude experimental condition feature
nocond = list(comp) + list(sup)
ocm_nocond = ocm[nocond]
ocm_nocond = pd.concat([ocm_nocond,ocm.iloc[:,-1]], axis=1)

In [10]:
desc = pd.read_csv('input/Descriptors.csv',skiprows = [0],index_col='symbol').drop(['Unnamed: 0',
                                                                               'name',
                                                                               'ionic radius',
                                                                               'covalent radius',
                                                                               'VdW radius',
                                                                               'crystal radius',
                                                                               'a x 106 ',
                                                                               'Heat capacity ',
                                                                               'l',
                                                                               'electron affinity ',
                                                                               'VE',
                                                                               'Surface energy '],axis=1)
desc = desc.loc[ocm.loc[:,:'Zr'].columns,]
desc = desc.fillna(desc.mean())

In [11]:
# num of elements in each multicomponent catalyst
print(Counter((ocm.loc[:,:'Zr']>0).sum(axis=1)))

Counter({2: 779, 3: 578, 1: 345, 4: 131})


## Fig.5

In [12]:
def scatter(pred,exp,xlabel='Yield_ex(%)',ylabel='Yield_pr(%)',Title='',col='blue',wt_value=True,save=False):
    range = [ocm.iloc[:,-1].min() ,ocm.iloc[:,-1].max() ]
    val0 = (pred - exp)**2
    str0 = 'RMSE: %1.3f' % math.sqrt(val0.mean())
    plt.figure(figsize=(3, 3), dpi=100)
    plt.plot(range, range, color='0.5')
    plt.scatter(exp,pred, s=10,
                facecolors=col, edgecolors=col, alpha=0.4)
    plt.xlim(range[0], range[1])
    plt.ylim(range[0], range[1])
    if wt_value:
        plt.text(range[1] - 0.97 * (range[1] - range[0]),
                 range[1] - 0.05 * (range[1] - range[0]), str0, fontsize=8.5)

    plt.xlabel(xlabel, fontsize=14)
    plt.ylabel(ylabel, fontsize=14)
    plt.title(Title, fontsize=15)
    #plt.legend(loc='lower right')

    if save :
        plt.savefig(Title.replace(' ','_') + 'one_shot' + '.png', format='png',
                    bbox_inches='tight', dpi=300)
        plt.savefig(Title.replace(' ','_') + 'one_shot' + '.pdf', format='pdf',
                    bbox_inches='tight', dpi=300)
        plt.close()

### ocm_nocond

In [13]:
cv_xgb.fit(ocm_nocond.iloc[:,:-1], ocm_nocond.iloc[:,-1])

cvdepth = cv_xgb.best_params_['max_depth']
cvsubsample = cv_xgb.best_params_['subsample']
cvcolsample = cv_xgb.best_params_['colsample_bytree']
cvlr = cv_xgb.best_params_['learning_rate']

print(cvdepth, cvsubsample, cvcolsample, cvlr)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/vers

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/

4 1 0.9 0.05


In [14]:
estimator = XGBRegressor(n_estimators=ntree,\
                       max_depth=cvdepth,\
                       subsample=cvsubsample,\
                       colsample_bytree=cvcolsample,\
                       learning_rate=cvlr,\
                       random_state=929, seed=929, n_jobs=-1)
print(estimator)

best_xgb_ocmnocond = estimator

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=0.9, gamma=0, importance_type='gain',
             learning_rate=0.05, max_delta_step=0, max_depth=4,
             min_child_weight=1, missing=None, n_estimators=10, n_jobs=-1,
             nthread=None, objective='reg:linear', random_state=929,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=929,
             silent=True, subsample=1)


In [15]:
train_test = shuffle(ocm_nocond, random_state=10)

In [16]:
cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10)
cv.cross_validation()

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 8.329 STD_test: 0.499
RMSE_train: 8.275 STD_train: 0.045
Data size:(1833, 69)


In [17]:
cv.plot_pred_expected(save=True,\
                      title='',\
                      os_title='',\
                      titlesize = 11, \
                      filename='output/fig5_onlycomposition_',
                      close=True)

(1650, 69) (1650,)
RMSE(Train): 8.277
RMSE(Test): 8.308


/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


### ocm

In [18]:
cv_xgb.fit(ocm.iloc[:,:-1], ocm.iloc[:,-1])

cvdepth = cv_xgb.best_params_['max_depth']
cvsubsample = cv_xgb.best_params_['subsample']
cvcolsample = cv_xgb.best_params_['colsample_bytree']
cvlr = cv_xgb.best_params_['learning_rate']

print(cvdepth, cvsubsample, cvcolsample, cvlr)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/vers

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/

4 1 0.9 0.1


In [19]:
estimator = XGBRegressor(n_estimators=ntree,\
                       max_depth=cvdepth,\
                       subsample=cvsubsample,\
                       colsample_bytree=cvcolsample,\
                       learning_rate=cvlr,\
                       random_state=929, seed=929, n_jobs=-1)
print(estimator)

best_xgb_ocm = estimator

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=0.9, gamma=0, importance_type='gain',
             learning_rate=0.1, max_delta_step=0, max_depth=4,
             min_child_weight=1, missing=None, n_estimators=10, n_jobs=-1,
             nthread=None, objective='reg:linear', random_state=929,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=929,
             silent=True, subsample=1)


In [20]:
train_test = shuffle(ocm, random_state=10)

In [21]:
cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10)
cv.cross_validation()

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 6.578 STD_test: 0.447
RMSE_train: 6.382 STD_train: 0.032
Data size:(1833, 104)


In [22]:
cv.plot_pred_expected(save=True,\
                      title='',\
                      os_title='',\
                      titlesize = 11, \
                      filename='output/fig5_composition&condition_',
                      close=True)

(1649, 104) (1649,)
RMSE(Train): 6.372
RMSE(Test): 6.597


/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


### ocm_desc

In [23]:
cv_xgb.fit(ocm_desc.iloc[:,:-1],ocm_desc.iloc[:,-1])

cvdepth = cv_xgb.best_params_['max_depth']
cvsubsample = cv_xgb.best_params_['subsample']
cvcolsample = cv_xgb.best_params_['colsample_bytree']
cvlr = cv_xgb.best_params_['learning_rate']

print(cvdepth, cvsubsample, cvcolsample, cvlr)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/vers

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/

4 0.9 1 0.05


In [24]:
estimator = XGBRegressor(n_estimators=ntree,\
                       max_depth=cvdepth,\
                       subsample=cvsubsample,\
                       colsample_bytree=cvcolsample,\
                       learning_rate=cvlr,\
                       random_state=929, seed=929, n_jobs=-1)
print(estimator)

best_xgb_ocmdesc = estimator

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, importance_type='gain',
             learning_rate=0.05, max_delta_step=0, max_depth=4,
             min_child_weight=1, missing=None, n_estimators=10, n_jobs=-1,
             nthread=None, objective='reg:linear', random_state=929,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=929,
             silent=True, subsample=0.9)


In [25]:
train_test = shuffle(ocm_desc, random_state=10)

In [26]:
cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10, redc=True)
cv.cross_validation()

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 8.148 STD_test: 0.481
RMSE_train: 8.034 STD_train: 0.041
Data size:(1833, 148)


In [27]:
xgb_cvbest = cv

In [28]:
cv.plot_pred_expected(save=True,\
                      title='',\
                      os_title='',\
                      filename='output/fig4-fig5_gb_',\
                      titlesize=11,
                      close=True)

(1650, 148) (1650,)
RMSE(Train): 8.002
RMSE(Test): 8.224


/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


## Fig.4

In [29]:
train_test = shuffle(ocm_desc, random_state=10)

In [30]:
# LASSO
cv_lasso.fit(ocm_desc.iloc[:,:-1],ocm_desc.iloc[:,-1])
cvalpha = cv_lasso.best_params_['alpha']
estimator = Lasso(random_state=929, alpha=cvalpha)

best_lasso_ocmdesc = estimator
print(estimator)

cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10, redc=True)
cv.cross_validation()
cv.plot_pred_expected(save=True,\
                      title='',\
                      os_title='',\
                      filename='output/fig4_lasso_', \
                      titlesize=11,
                      close=True)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17197.705109852704, tolerance: 5.5818553610883805
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17097.472180321434, tolerance: 5.508298215425533
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=929,
      selection='cyclic', tol=0.0001, warm_start=False)


/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25163.081208441778, tolerance: 7.544120143723469
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25022.687105336274, tolerance: 7.431372534869618
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25715.04039060453, tolerance: 7.516430010915706
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordin

RMSE_test: 5.987 STD_test: 0.512
RMSE_train: 5.402 STD_train: 0.049
Data size:(1833, 148)
(1650, 148) (1650,)
RMSE(Train): 5.381
RMSE(Test): 6.059


/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 24689.096255673692, tolerance: 7.377854583181819
  positive)


In [31]:
# Ridge
cv_ridge.fit(ocm_desc.iloc[:,:-1],ocm_desc.iloc[:,-1])
cvalpha = cv_ridge.best_params_['alpha']
estimator = Ridge(random_state=929, alpha=cvalpha)

best_ridge_ocmdesc = estimator
print(estimator)

cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10, redc=True)
cv.cross_validation()
cv.plot_pred_expected(save=True,\
                      title='',\
                      os_title='',\
                      filename='output/fig4_ridge_',\
                      titlesize=11,
                      close=True)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=929, solver='auto', tol=0.001)
RMSE_test: 6.338 STD_test: 0.546
RMSE_train: 5.356 STD_train: 0.049
Data size:(1833, 148)
(1649, 148) (1649,)
RMSE(Train): 5.453
RMSE(Test): 6.345


In [32]:
# KRR
cv_krr.fit(ocm_desc.iloc[:,:-1],ocm_desc.iloc[:,-1])
cvalpha = cv_krr.best_params_['alpha']
cvgamma = cv_krr.best_params_['gamma']
estimator = KernelRidge(kernel='rbf',alpha=cvalpha, gamma=cvgamma)

best_krr_ocmdesc = estimator
print(estimator)

cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10, redc=True)
cv.cross_validation()
cv.plot_pred_expected(save=True,\
                      title='', \
                      os_title='',\
                      filename='output/fig4_krr_',\
                      titlesize=11,
                      close=True)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


KernelRidge(alpha=0.1, coef0=1, degree=3, gamma=0.1, kernel='rbf',
            kernel_params=None)
RMSE_test: 10.857 STD_test: 0.571
RMSE_train: 1.063 STD_train: 0.007
Data size:(1833, 148)
(1650, 148) (1650,)
RMSE(Train): 1.054
RMSE(Test): 10.910


In [33]:
# SVR
cv_svr.fit(ocm_desc.iloc[:,:-1], ocm_desc.iloc[:,-1])
cvC = cv_svr.best_params_['C']
cvgamma = cv_svr.best_params_['gamma']
cvepsilon = cv_svr.best_params_['epsilon']
estimator = SVR(kernel='rbf', epsilon=cvepsilon, C=cvC, gamma=cvgamma)

best_svr_ocmdesc = estimator
print(estimator)

cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10, redc=True)
cv.cross_validation()
cv.plot_pred_expected(save=True,\
                      title='',\
                      os_title='',\
                      filename='output/fig4_svr_',\
                      titlesize=11,
                      close=True)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
RMSE_test: 6.415 STD_test: 0.308
RMSE_train: 1.510 STD_train: 0.052
Data size:(1833, 148)
(1649, 148) (1649,)
RMSE(Train): 1.533
RMSE(Test): 6.468


In [34]:
# RFR
estimator = RandomForestRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

best_rfr_ocmdesc = estimator
print(estimator)

cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10, redc=True)
cv.cross_validation()
cv.plot_pred_expected(save=True,\
                      title='',\
                      os_title='',\
                      filename='output/fig4_rfr_',\
                      titlesize=11,
                      close=True)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                      oob_score=False, random_state=929, verbose=0,
                      warm_start=False)
RMSE_test: 4.671 STD_test: 0.287
RMSE_train: 1.974 STD_train: 0.052
Data size:(1833, 148)
(1650, 148) (1650,)
RMSE(Train): 2.012
RMSE(Test): 4.601


In [35]:
# ETR
estimator = ExtraTreesRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

best_etr_ocmdesc = estimator
print(estimator)

cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10, redc=True)
cv.cross_validation()
cv.plot_pred_expected(save=True,\
                      title='',\
                      os_title='',\
                      filename='output/fig4_etr_',\
                      titlesize=11,
                      close=True)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
                    max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                    oob_score=False, random_state=929, verbose=0,
                    warm_start=False)
RMSE_test: 4.508 STD_test: 0.358
RMSE_train: 0.163 STD_train: 0.018
Data size:(1833, 148)
(1650, 148) (1650,)
RMSE(Train): 0.173
RMSE(Test): 4.435


## Fig.6 (A)

In [36]:
plt.figure(figsize=(6, 4), dpi=300)
xgb_cvbest.importance.mean(axis=1).sort_values(ascending=False).iloc[:20][::-1].plot(kind='barh',color='b')
plt.savefig('output/fig6_ocm.png', format='png',bbox_inches='tight', dpi=300)
plt.savefig('output/fig6_ocm.pdf', format='pdf',bbox_inches='tight', dpi=300)
plt.close()

## Fig.2 (A)

In [37]:
common = []
tmp = ocm.loc[:,:'Zr']
count = 0
for key, row in tmp.iterrows():
    for i, v in tmp.iloc[count + 1:].iterrows():
        temp = (np.array(row) > 1e-5)
        temp2 = (np.array(v) > 1e-5)
        val = (temp * temp2).sum()
        common.append(val)  
    count = count + 1

freq = dict(Counter(common))
plt.figure(figsize=(2.5, 3))
plt.bar(freq.keys(), freq.values())
plt.xlabel('# Common Elements')
plt.ylabel('Frequency (log)')
plt.yscale('log')
plt.savefig('output/fig2_ocm.png',format='png', bbox_inches='tight',dpi=300)
plt.savefig('output/fig2_ocm.pdf',format='pdf', bbox_inches='tight',dpi=300)
plt.close()

In [38]:
ocm_orig = pd.read_csv("input/OCM_matrix.csv").drop(['Unnamed: 0'], axis=1)
val = (ocm_orig.loc[:,:'Zr'] > 0).sum(axis=1).mean()
print('%1.2f' % val)

2.30


In [39]:
dat = (ocm_orig.loc[:,:'Zr'] > 0).sum(axis=0).sort_values()
ind = dat < 10
print(dat[ind])

I     1
Ar    1
Au    1
Ga    1
In    2
Te    2
Rh    2
Pd    2
Lu    2
Sc    2
Ru    2
Be    3
Re    3
Cd    3
Cr    3
Ge    3
N     4
Tm    4
Pt    4
Ho    6
Er    6
Dy    8
Cu    8
Tb    9
dtype: int64


## Fig.3 & Table 3

In [40]:
def crossvalid(xx,yy,model,cv,txt):
    cv = Cv_Pred_Expected(estimator=model,X=xx,y=yy,cv=10,redc = True)
    cv.cross_validation()
    tes_rmse = cv.rmse
    tes_std = cv.std
    trn_rmse = cv.rmse_train
    trn_std = cv.std_train
    print()
    print("[%s] RMSE %1.3f (STD: %1.3f) ... test" % (txt,tes_rmse,tes_std))
    print("[%s] RMSE %1.3f (STD: %1.3f) ... train" % (txt,trn_rmse,trn_std))
    ret_obj = {}
    ret_obj['tes_mean'] = tes_rmse
    ret_obj['tes_sd']   = tes_std 
    ret_obj['trn_mean'] = trn_rmse
    ret_obj['trn_sd']   = trn_std 
    return ret_obj

### ocm_nocond

In [41]:
rmse_ocm_nocond = {}
data = shuffle(ocm_nocond, random_state=10)

target_data = ocm_nocond

In [42]:
# Lasso
cv_lasso.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_lasso.best_params_['alpha']
best_lasso = Lasso(random_state=929, alpha=cvalpha)

# Ridge
cv_ridge.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_ridge.best_params_['alpha']
best_ridge = Ridge(random_state=929, alpha=cvalpha)

# KRR
cv_krr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_krr.best_params_['alpha']
cvgamma = cv_krr.best_params_['gamma']
best_krr = KernelRidge(kernel='rbf', alpha=cvalpha, gamma=cvgamma)

# SVR
cv_svr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvC = cv_svr.best_params_['C']
cvgamma = cv_svr.best_params_['gamma']
cvepsilon = cv_svr.best_params_['epsilon']
best_svr = SVR(kernel='rbf', epsilon=cvepsilon, C=cvC, gamma=cvgamma)

# RFR
best_rfr = RandomForestRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# ETR
best_etr = ExtraTreesRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default

In [43]:
model = best_lasso
print(model)
rmse_ocm_nocond['Lasso'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Lasso')

model = best_ridge
print(model)
rmse_ocm_nocond['Ridge'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10,  'Ridge')

model = best_krr
print(model)
rmse_ocm_nocond['KRR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'KRR')

model = best_svr
print(model)
rmse_ocm_nocond['SVR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'SVR')

model = best_rfr
print(model)
rmse_ocm_nocond['RFR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'RFR')

model = best_etr
print(model)
rmse_ocm_nocond['ETR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'ETR')

model = best_xgb_ocmnocond
print(model)
rmse_ocm_nocond['XGB'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'XGB')

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=929,
      selection='cyclic', tol=0.0001, warm_start=False)
RMSE_test: 6.318 STD_test: 0.426
RMSE_train: 6.105 STD_train: 0.052
Data size:(1833, 69)

[Lasso] RMSE 6.318 (STD: 0.426) ... test
[Lasso] RMSE 6.105 (STD: 0.052) ... train
Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=929, solver='auto', tol=0.001)
RMSE_test: 6.840 STD_test: 1.149
RMSE_train: 6.085 STD_train: 0.052
Data size:(1833, 69)

[Ridge] RMSE 6.840 (STD: 1.149) ... test
[Ridge] RMSE 6.085 (STD: 0.052) ... train
KernelRidge(alpha=0.1, coef0=1, degree=3, gamma=0.1, kernel='rbf',
            kernel_params=None)
RMSE_test: 8.645 STD_test: 0.525
RMSE_train: 2.467 STD_train: 0.033
Data size:(1833, 69)

[KRR] RMSE 8.645 (STD: 0.525) ... test
[KRR] RMSE 2.467 (STD: 0.033) ... train
SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 8.329 STD_test: 0.499
RMSE_train: 8.275 STD_train: 0.045
Data size:(1833, 69)

[XGB] RMSE 8.329 (STD: 0.499) ... test
[XGB] RMSE 8.275 (STD: 0.045) ... train


### ocm

In [44]:
rmse_ocm = {}
data = shuffle(ocm, random_state=10)
target_data = ocm

In [45]:
# Lasso
cv_lasso.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_lasso.best_params_['alpha']
best_lasso = Lasso(random_state=929, alpha=cvalpha)

# Ridge
cv_ridge.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_ridge.best_params_['alpha']
best_ridge = Ridge(random_state=929, alpha=cvalpha)

# KRR
cv_krr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_krr.best_params_['alpha']
cvgamma = cv_krr.best_params_['gamma']
best_krr = KernelRidge(kernel='rbf', alpha=cvalpha, gamma=cvgamma)

# SVR
cv_svr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvC = cv_svr.best_params_['C']
cvgamma = cv_svr.best_params_['gamma']
cvepsilon = cv_svr.best_params_['epsilon']
best_svr = SVR(kernel='rbf', epsilon=cvepsilon, C=cvC, gamma=cvgamma)

# RFR
best_rfr = RandomForestRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# ETR
best_etr = ExtraTreesRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 300.2468069471979, tolerance: 5.508298215425533
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning:

In [46]:
model = best_lasso
print(model)
rmse_ocm['Lasso'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Lasso')

model = best_ridge
print(model)
rmse_ocm['Ridge'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Ridge')

model = best_krr
print(model)
rmse_ocm['KRR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'KRR')

model = best_svr
print(model)
rmse_ocm['SVR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'SVR')

model = best_rfr
print(model)
rmse_ocm['RFR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'RFR')

model = best_etr
print(model)
rmse_ocm['ETR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'ETR')

model = best_xgb_ocm
print(model)
rmse_ocm['XGB'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'XGB')

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=929,
      selection='cyclic', tol=0.0001, warm_start=False)


/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1057.6749693350648, tolerance: 7.544120143723469
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 712.215468669383, tolerance: 7.431372534869618
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 72.41255264348729, tolerance: 7.516430010915706
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinat

RMSE_test: 6.069 STD_test: 0.539
RMSE_train: 5.590 STD_train: 0.049
Data size:(1833, 104)

[Lasso] RMSE 6.069 (STD: 0.539) ... test
[Lasso] RMSE 5.590 (STD: 0.049) ... train
Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=929, solver='auto', tol=0.001)
RMSE_test: 6.641 STD_test: 0.823
RMSE_train: 5.561 STD_train: 0.049
Data size:(1833, 104)

[Ridge] RMSE 6.641 (STD: 0.823) ... test
[Ridge] RMSE 5.561 (STD: 0.049) ... train
KernelRidge(alpha=0.1, coef0=1, degree=3, gamma=0.1, kernel='rbf',
            kernel_params=None)
RMSE_test: 9.829 STD_test: 0.543
RMSE_train: 1.185 STD_train: 0.027
Data size:(1833, 104)

[KRR] RMSE 9.829 (STD: 0.543) ... test
[KRR] RMSE 1.185 (STD: 0.027) ... train
SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
RMSE_test: 6.163 STD_test: 0.368
RMSE_train: 1.893 STD_train: 0.058
Data size:(1833, 104)

[SVR] RMSE 6.

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 6.578 STD_test: 0.447
RMSE_train: 6.382 STD_train: 0.032
Data size:(1833, 104)

[XGB] RMSE 6.578 (STD: 0.447) ... test
[XGB] RMSE 6.382 (STD: 0.032) ... train


### ocm_desc

In [47]:
rmse_ocm_desc = {}
data = shuffle(ocm_desc, random_state=929)
target_data = ocm_desc

In [48]:
model = best_lasso_ocmdesc
print(model)
rmse_ocm_desc['Lasso'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Lasso')

model = best_ridge_ocmdesc
print(model)
rmse_ocm_desc['Ridge'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Ridge')

model = best_krr_ocmdesc
print(model)
rmse_ocm_desc['KRR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'KRR')

model = best_svr_ocmdesc
print(model)
rmse_ocm_desc['SVR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'SVR')

model = best_rfr_ocmdesc
print(model)
rmse_ocm_desc['RFR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'RFR')

model = best_etr_ocmdesc
print(model)
rmse_ocm_desc['ETR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'ETR')

model = best_xgb_ocmdesc
print(model)
rmse_ocm_desc['XGB'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'XGB')

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=929,
      selection='cyclic', tol=0.0001, warm_start=False)


/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 24281.24615518106, tolerance: 7.365366804123711
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25443.1501925046, tolerance: 7.531452939357186
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 24131.01359004105, tolerance: 7.383119440873256
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate

RMSE_test: 6.038 STD_test: 0.608
RMSE_train: 5.401 STD_train: 0.056
Data size:(1833, 148)

[Lasso] RMSE 6.038 (STD: 0.608) ... test
[Lasso] RMSE 5.401 (STD: 0.056) ... train
Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=929, solver='auto', tol=0.001)
RMSE_test: 6.499 STD_test: 0.627
RMSE_train: 5.354 STD_train: 0.057
Data size:(1833, 148)

[Ridge] RMSE 6.499 (STD: 0.627) ... test
[Ridge] RMSE 5.354 (STD: 0.057) ... train
KernelRidge(alpha=0.1, coef0=1, degree=3, gamma=0.1, kernel='rbf',
            kernel_params=None)
RMSE_test: 10.904 STD_test: 0.552
RMSE_train: 1.062 STD_train: 0.009
Data size:(1833, 148)

[KRR] RMSE 10.904 (STD: 0.552) ... test
[KRR] RMSE 1.062 (STD: 0.009) ... train
SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
RMSE_test: 6.421 STD_test: 0.301
RMSE_train: 1.508 STD_train: 0.049
Data size:(1833, 148)

[SVR] RMSE 6

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 8.130 STD_test: 0.492
RMSE_train: 8.039 STD_train: 0.039
Data size:(1833, 148)

[XGB] RMSE 8.130 (STD: 0.492) ... test
[XGB] RMSE 8.039 (STD: 0.039) ... train


### Fig.3

In [49]:
methods = ['Lasso','Ridge','KRR','SVR','RFR','ETR','XGB']

tes_mean = [rmse_ocm_desc[m]['tes_mean'] for m in methods]
tes_sd   = [rmse_ocm_desc[m]['tes_sd']   for m in methods]
trn_mean = [rmse_ocm_desc[m]['trn_mean'] for m in methods]
trn_sd   = [rmse_ocm_desc[m]['trn_sd']   for m in methods]

# 2*sigma = 95% CI
tes_ci = 2 * np.array(tes_sd)
trn_ci = 2 * np.array(trn_sd)

plt.figure(figsize=(6,3))

ind = np.arange(len(methods))
width = 0.4
plt.bar(ind, trn_mean, width, color='0.9', align='center', yerr=trn_ci)
plt.bar(ind+width, tes_mean, width, color='0.6', align='center', yerr=tes_ci)

plt.ylabel('RMSE (%)')
plt.xticks(ind+width/2,methods,rotation=45, fontsize=12)
plt.legend(('Training Error',
            'Test Error'), loc ="upper right", prop={'size':12})
plt.ylim(0,np.max(tes_mean)+5)

for x,y in zip(ind, trn_mean):
    plt.text(x, y+1.5, '%.2f' % y, ha='center', va= 'bottom', rotation='90', fontsize=10)

for x,y in zip(ind+width, tes_mean):
    plt.text(x, y+1.5, '%.2f' % y, ha='center', va= 'bottom', rotation='90', fontsize=10)

plt.savefig('output/fig3.png',format='png', bbox_inches='tight',dpi=300)
plt.savefig('output/fig3.pdf',format='pdf', bbox_inches='tight',dpi=300)
plt.close()

### Table 3

In [50]:
pd.options.display.precision = 2

print(pd.DataFrame(rmse_ocm_nocond))

print(pd.DataFrame(rmse_ocm))

print(pd.DataFrame(rmse_ocm_desc))

          Lasso  Ridge   KRR   SVR   RFR   ETR   XGB
tes_mean   6.32   6.84  8.65  6.00  5.45  5.55  8.33
tes_sd     0.43   1.15  0.52  0.30  0.45  0.30  0.50
trn_mean   6.11   6.09  2.47  2.60  2.75  1.85  8.27
trn_sd     0.05   0.05  0.03  0.05  0.03  0.03  0.05
          Lasso  Ridge   KRR   SVR   RFR   ETR   XGB
tes_mean   6.07   6.64  9.83  6.16  4.62  4.52  6.58
tes_sd     0.54   0.82  0.54  0.37  0.35  0.28  0.45
trn_mean   5.59   5.56  1.18  1.89  1.98  0.16  6.38
trn_sd     0.05   0.05  0.03  0.06  0.04  0.02  0.03
          Lasso  Ridge       KRR   SVR   RFR   ETR   XGB
tes_mean   6.04   6.50  1.09e+01  6.42  4.59  4.45  8.13
tes_sd     0.61   0.63  5.52e-01  0.30  0.31  0.47  0.49
trn_mean   5.40   5.35  1.06e+00  1.51  1.97  0.16  8.04
trn_sd     0.06   0.06  9.44e-03  0.05  0.04  0.02  0.04


## WGS data

In [51]:
desc = pd.read_csv('input/Descriptors.csv',skiprows = [0],index_col='symbol').drop(['Unnamed: 0',
                                                                               'name',
                                                                               'ionic radius',
                                                                               'covalent radius',
                                                                               'VdW radius',
                                                                               'crystal radius',
                                                                               'a x 106 ',
                                                                               'Heat capacity ',
                                                                               'l',
                                                                               'electron affinity ',
                                                                               'VE',
                                                                               'Surface energy '],axis=1)
desc = desc.iloc[:83, :]

In [52]:
wgs = pd.read_csv("input/wgs.csv")
wgs.index = list(wgs.iloc[:,0])
wgs = wgs.drop([wgs.columns[0]], axis=1)

wgs_desc = pd.read_csv("input/wgs_desc.csv")
wgs_desc.index = list(wgs_desc.iloc[:,0])
wgs_desc = wgs_desc.drop([wgs_desc.columns[0]], axis=1)

atom = list(wgs.loc[:,:'Pd'].columns) + list(wgs.loc[:,'Li':'Sr'].columns)
desc = desc.loc[atom]

desc=desc.fillna(desc.mean())

In [53]:
# WGS
ntree = 1500
if debug_run:
    ntree = 10

# Lasso
cv_lasso = GridSearchCV(
    Lasso(random_state=929), cv=5,
    param_grid={"alpha": range_lasso})

# Ridge
cv_ridge = GridSearchCV(
    Ridge(random_state=929), cv=5,
    param_grid={"alpha": range_ridge})

# KRR
cv_krr = GridSearchCV(
    KernelRidge(kernel='rbf'), cv=5,
    param_grid={"alpha": range_krr_alpha, 
                "gamma": range_krr_gamma})

# SVR
cv_svr = GridSearchCV(
    SVR(kernel='rbf'), cv=5,
    param_grid={"C": range_svr_C, 
                "gamma": range_svr_gamma, 
                "epsilon": range_svr_epsilon})

# XGB
cv_xgb = GridSearchCV(
    XGBRegressor(random_state=929,
                 n_estimators=ntree), cv=5,
    param_grid={"max_depth": range_depth, 
                "subsample": range_subsample, 
                "colsample_bytree": range_colsample,
                "learning_rate": range_lr})

## Fig.6 (B)

In [54]:
cv_xgb.fit(wgs_desc.iloc[:,:-1], wgs_desc.iloc[:,-1])

cvdepth = cv_xgb.best_params_['max_depth']
cvsubsample = cv_xgb.best_params_['subsample']
cvcolsample = cv_xgb.best_params_['colsample_bytree']
cvlr = cv_xgb.best_params_['learning_rate']

print(cvdepth, cvsubsample, cvcolsample, cvlr)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/vers

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/

3 0.9 1 0.05


In [55]:
estimator = XGBRegressor(n_estimators=ntree,\
                       max_depth=cvdepth,\
                       subsample=cvsubsample,\
                       colsample_bytree=cvcolsample,\
                       learning_rate=cvlr,\
                       random_state=929, seed=929, n_jobs=-1)
print(estimator)

best_xgb_wgsdesc = estimator

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, importance_type='gain',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=None, n_estimators=10, n_jobs=-1,
             nthread=None, objective='reg:linear', random_state=929,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=929,
             silent=True, subsample=0.9)


In [56]:
train_test = shuffle(wgs_desc, random_state=929)

In [57]:
cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10, redc=True)
cv.cross_validation()

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 37.750 STD_test: 1.484
RMSE_train: 37.652 STD_train: 0.109
Data size:(4185, 134)


In [58]:
xgb_cvbest = cv

In [59]:
cv.plot_pred_expected(save=True,\
                      title='',\
                      os_title='',\
                      filename='output/wgs_gb_',\
                      titlesize=11,
                      close=True)

(3767, 134) (3767,)
RMSE(Train): 37.699
RMSE(Test): 37.696


/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [60]:
plt.figure(figsize=(6, 4), dpi=300)
xgb_cvbest.importance.mean(axis=1).sort_values(ascending=False).iloc[:20][::-1].plot(kind='barh',color='b')
plt.savefig('output/fig6_wgs.png', format='png',bbox_inches='tight', dpi=300)
plt.savefig('output/fig6_wgs.pdf', format='pdf',bbox_inches='tight', dpi=300)
plt.close()

## Fig.2 (B)

In [61]:
common = []
hoge = wgs.ix[:,'Pt':'Pd']
count = 0
for key, row in hoge.iterrows():
    for i, v in hoge.iloc[count + 1:].iterrows():
        temp = (np.array(row) > 1e-5)
        temp2 = (np.array(v) > 1e-5)
        val = (temp * temp2).sum()
        common.append(val)     
    count = count + 1

freq = dict(Counter(common))
plt.figure(figsize=(1.5, 3))
plt.bar(freq.keys(), freq.values())
plt.xlabel('# Common Elements')
plt.ylabel('Frequency (log)')
plt.yscale('log')
plt.savefig('output/fig2_wgs.png',format='png', bbox_inches='tight',dpi=300)
plt.savefig('output/fig2_wgs.pdf',format='pdf', bbox_inches='tight',dpi=300)
plt.close()

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


## Table 4 (wgs)

### wgs_nocond

In [62]:
comppro = list(wgs.loc[:, :'Pd'].columns)
wgs_nocond = wgs.loc[:, comppro + ['CO Conversion']]

In [63]:
rmse_wgs_nocond = {}
data = shuffle(wgs_nocond, random_state=10)
target_data = wgs_nocond

In [64]:
# Lasso
cv_lasso.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_lasso.best_params_['alpha']
best_lasso = Lasso(random_state=929, alpha=cvalpha)

# Ridge
cv_ridge.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_ridge.best_params_['alpha']
best_ridge = Ridge(random_state=929, alpha=cvalpha)

# KRR
cv_krr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_krr.best_params_['alpha']
cvgamma = cv_krr.best_params_['gamma']
best_krr = KernelRidge(kernel='rbf', alpha=cvalpha, gamma=cvgamma)

# SVR
cv_svr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvC = cv_svr.best_params_['C']
cvgamma = cv_svr.best_params_['gamma']
cvepsilon = cv_svr.best_params_['epsilon']
best_svr = SVR(kernel='rbf', epsilon=cvepsilon, C=cvC, gamma=cvgamma)

# RFR
best_rfr = RandomForestRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# ETR
best_etr = ExtraTreesRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# XGB
cv_xgb.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvdepth = cv_xgb.best_params_['max_depth']
cvsubsample = cv_xgb.best_params_['subsample']
cvcolsample = cv_xgb.best_params_['colsample_bytree']
cvlr = cv_xgb.best_params_['learning_rate']
best_xgb = XGBRegressor(n_estimators=ntree,\
                       max_depth=cvdepth,\
                       subsample=cvsubsample,\
                       colsample_bytree=cvcolsample,\
                       learning_rate=cvlr,\
                       random_state=929, seed=929, n_jobs=-1)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/

In [65]:
model = best_lasso
print(model)
rmse_wgs_nocond['Lasso'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Lasso')

model = best_ridge
print(model)
rmse_wgs_nocond['Ridge'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10,  'Ridge')

model = best_krr
print(model)
rmse_wgs_nocond['KRR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'KRR')

model = best_svr
print(model)
rmse_wgs_nocond['SVR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'SVR')

model = best_rfr
print(model)
rmse_wgs_nocond['RFR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'RFR')

model = best_etr
print(model)
rmse_wgs_nocond['ETR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'ETR')

model = best_xgb
print(model)
rmse_wgs_nocond['XGB'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'XGB')

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=929,
      selection='cyclic', tol=0.0001, warm_start=False)
RMSE_test: 32.271 STD_test: 0.494
RMSE_train: 32.066 STD_train: 0.053
Data size:(4185, 35)

[Lasso] RMSE 32.271 (STD: 0.494) ... test
[Lasso] RMSE 32.066 (STD: 0.053) ... train
Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=929, solver='auto', tol=0.001)
RMSE_test: 32.301 STD_test: 0.454
RMSE_train: 31.993 STD_train: 0.052
Data size:(4185, 35)

[Ridge] RMSE 32.301 (STD: 0.454) ... test
[Ridge] RMSE 31.993 (STD: 0.052) ... train
KernelRidge(alpha=1.0, coef0=1, degree=3, gamma=0.1, kernel='rbf',
            kernel_params=None)
RMSE_test: 31.585 STD_test: 0.621
RMSE_train: 30.936 STD_train: 0.061
Data size:(4185, 35)

[KRR] RMSE 31.585 (STD: 0.621) ... test
[KRR] RMSE 30.936 (STD: 0.061) ... train
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 40.941 STD_test: 1.449
RMSE_train: 40.886 STD_train: 0.111
Data size:(4185, 35)

[XGB] RMSE 40.941 (STD: 1.449) ... test
[XGB] RMSE 40.886 (STD: 0.111) ... train


### wgs

In [66]:
rmse_wgs = {}
data = shuffle(wgs, random_state=10)
target_data = wgs

In [67]:
# Lasso
cv_lasso.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_lasso.best_params_['alpha']
best_lasso = Lasso(random_state=929, alpha=cvalpha)

# Ridge
cv_ridge.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_ridge.best_params_['alpha']
best_ridge = Ridge(random_state=929, alpha=cvalpha)

# KRR
cv_krr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_krr.best_params_['alpha']
cvgamma = cv_krr.best_params_['gamma']
best_krr = KernelRidge(kernel='rbf', alpha=cvalpha, gamma=cvgamma)

# SVR
cv_svr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvC = cv_svr.best_params_['C']
cvgamma = cv_svr.best_params_['gamma']
cvepsilon = cv_svr.best_params_['epsilon']
best_svr = SVR(kernel='rbf', epsilon=cvepsilon, C=cvC, gamma=cvgamma)

# RFR
best_rfr = RandomForestRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# ETR
best_etr = ExtraTreesRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# XGB
cv_xgb.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvdepth = cv_xgb.best_params_['max_depth']
cvsubsample = cv_xgb.best_params_['subsample']
cvcolsample = cv_xgb.best_params_['colsample_bytree']
cvlr = cv_xgb.best_params_['learning_rate']
best_xgb = XGBRegressor(n_estimators=ntree,\
                       max_depth=cvdepth,\
                       subsample=cvsubsample,\
                       colsample_bytree=cvcolsample,\
                       learning_rate=cvlr,\
                       random_state=929, seed=929, n_jobs=-1)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/

In [68]:
model = best_lasso
print(model)
rmse_wgs['Lasso'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Lasso')

model = best_ridge
print(model)
rmse_wgs['Ridge'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Ridge')

model = best_krr
print(model)
rmse_wgs['KRR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'KRR')

model = best_svr
print(model)
rmse_wgs['SVR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'SVR')

model = best_rfr
print(model)
rmse_wgs['RFR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'RFR')

model = best_etr
print(model)
rmse_wgs['ETR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'ETR')

model = best_xgb
print(model)
rmse_wgs['XGB'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'XGB')

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=929,
      selection='cyclic', tol=0.0001, warm_start=False)
RMSE_test: 24.059 STD_test: 0.692
RMSE_train: 23.711 STD_train: 0.076
Data size:(4185, 79)

[Lasso] RMSE 24.059 (STD: 0.692) ... test
[Lasso] RMSE 23.711 (STD: 0.076) ... train
Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=929, solver='auto', tol=0.001)
RMSE_test: 23.510 STD_test: 0.581
RMSE_train: 23.003 STD_train: 0.068
Data size:(4185, 79)

[Ridge] RMSE 23.510 (STD: 0.581) ... test
[Ridge] RMSE 23.003 (STD: 0.068) ... train
KernelRidge(alpha=0.1, coef0=1, degree=3, gamma=0.1, kernel='rbf',
            kernel_params=None)
RMSE_test: 35.989 STD_test: 1.028
RMSE_train: 6.412 STD_train: 0.089
Data size:(4185, 79)

[KRR] RMSE 35.989 (STD: 1.028) ... test
[KRR] RMSE 6.412 (STD: 0.089) ... train
SVR(C=10.0, cache_size=200, coef0=0.0, degree=3,

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 28.076 STD_test: 0.818
RMSE_train: 27.693 STD_train: 0.063
Data size:(4185, 79)

[XGB] RMSE 28.076 (STD: 0.818) ... test
[XGB] RMSE 27.693 (STD: 0.063) ... train


### wgs_desc

In [69]:
rmse_wgs_desc = {}
data = shuffle(wgs_desc, random_state=929)
target_data = wgs_desc

In [70]:
# Lasso
cv_lasso.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_lasso.best_params_['alpha']
best_lasso = Lasso(random_state=929, alpha=cvalpha)

# Ridge
cv_ridge.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_ridge.best_params_['alpha']
best_ridge = Ridge(random_state=929, alpha=cvalpha)

# KRR
cv_krr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_krr.best_params_['alpha']
cvgamma = cv_krr.best_params_['gamma']
best_krr = KernelRidge(kernel='rbf', alpha=cvalpha, gamma=cvgamma)

# SVR
cv_svr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvC = cv_svr.best_params_['C']
cvgamma = cv_svr.best_params_['gamma']
cvepsilon = cv_svr.best_params_['epsilon']
best_svr = SVR(kernel='rbf', epsilon=cvepsilon, C=cvC, gamma=cvgamma)

# RFR
best_rfr = RandomForestRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# ETR
best_etr = ExtraTreesRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 677317.0302980654, tolerance: 279.59831767828445
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 763897.3246697523, tolerance: 304.69124376603196
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.

In [71]:
model = best_lasso
print(model)
rmse_wgs_desc['Lasso'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Lasso')

model = best_ridge
print(model)
rmse_wgs_desc['Ridge'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Ridge')

model = best_krr
print(model)
rmse_wgs_desc['KRR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'KRR')

model = best_svr
print(model)
rmse_wgs_desc['SVR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'SVR')

model = best_rfr
print(model)
rmse_wgs_desc['RFR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'RFR')

model = best_etr
print(model)
rmse_wgs_desc['ETR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'ETR')

model = best_xgb_wgsdesc
print(model)
rmse_wgs_desc['XGB'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'XGB')

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=929,
      selection='cyclic', tol=0.0001, warm_start=False)


/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1134421.5566300275, tolerance: 402.66865098908943
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1124976.221860943, tolerance: 403.0522811518897
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1134714.3029311716, tolerance: 403.2200847103915
  positive)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/linear_model/coordi

RMSE_test: 24.039 STD_test: 0.816
RMSE_train: 23.642 STD_train: 0.106
Data size:(4185, 134)

[Lasso] RMSE 24.039 (STD: 0.816) ... test
[Lasso] RMSE 23.642 (STD: 0.106) ... train
Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=929, solver='auto', tol=0.001)
RMSE_test: 23.487 STD_test: 0.916
RMSE_train: 22.944 STD_train: 0.105
Data size:(4185, 134)

[Ridge] RMSE 23.487 (STD: 0.916) ... test
[Ridge] RMSE 22.944 (STD: 0.105) ... train
KernelRidge(alpha=0.1, coef0=1, degree=3, gamma=0.1, kernel='rbf',
            kernel_params=None)
RMSE_test: 45.596 STD_test: 1.391
RMSE_train: 5.787 STD_train: 0.067
Data size:(4185, 134)

[KRR] RMSE 45.596 (STD: 1.391) ... test
[KRR] RMSE 5.787 (STD: 0.067) ... train
SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
RMSE_test: 30.480 STD_test: 0.681
RMSE_train: 22.744 STD_train: 0.075
Data size:(4185, 134)

[S

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 37.750 STD_test: 1.484
RMSE_train: 37.652 STD_train: 0.109
Data size:(4185, 134)

[XGB] RMSE 37.750 (STD: 1.484) ... test
[XGB] RMSE 37.652 (STD: 0.109) ... train


### Table 4

In [72]:
pd.options.display.precision = 2

print(pd.DataFrame(rmse_wgs_nocond))

print(pd.DataFrame(rmse_wgs))

print(pd.DataFrame(rmse_wgs_desc))

          Lasso  Ridge    KRR    SVR    RFR    ETR    XGB
tes_mean  32.27  32.30  31.59  32.15  31.29  31.17  40.94
tes_sd     0.49   0.45   0.62   0.77   0.67   0.64   1.45
trn_mean  32.07  31.99  30.94  31.94  29.70  29.61  40.89
trn_sd     0.05   0.05   0.06   0.07   0.07   0.07   0.11
          Lasso  Ridge    KRR    SVR    RFR       ETR    XGB
tes_mean  24.06  23.51  35.99  26.83  12.24  1.10e+01  28.08
tes_sd     0.69   0.58   1.03   0.52   0.94  9.11e-01   0.82
trn_mean  23.71  23.00   6.41  20.13   5.26  1.15e-02  27.69
trn_sd     0.08   0.07   0.09   0.08   0.13  5.06e-03   0.06
          Lasso  Ridge    KRR    SVR    RFR    ETR    XGB
tes_mean  24.04  23.49  45.60  30.48  12.19  10.85  37.75
tes_sd     0.82   0.92   1.39   0.68   0.88   0.96   1.48
trn_mean  23.64  22.94   5.79  22.74   5.08   0.02  37.65
trn_sd     0.11   0.11   0.07   0.07   0.08   0.02   0.11


## CO oxidation data

In [73]:
co = pd.read_csv('input/co.csv')
ind = co['Data No']
co.index = ind
co= co.drop(['Data No'], axis=1)

co_desc = pd.read_csv('input/co_desc.csv')
ind = co_desc['Data No']
co_desc.index = ind
co_desc= co_desc.drop(['Data No'], axis=1)

In [74]:
# devine to each groups
baseatom = co.loc[:,:'Pd'].columns
supatom = co.loc[:,'Al_s':'Co_s'].columns
pro = co.loc[:,'Ce_p':'Cu_p']
proatom = [re.findall(r'[A-Z][a-z]?',x) for x in co.loc[:,'Ce_p':'Cu_p']]
proatom = [y for x in proatom for y in x ]
env = co.drop(list(baseatom)+list(pro),axis=1).iloc[:,:-1].columns

conv = co.iloc[:,-1]

In [75]:
desc = pd.read_csv('input/Descriptors.csv',skiprows = [0],index_col='symbol').drop(['Unnamed: 0',
                                                                               'name',
                                                                               'ionic radius',
                                                                               'covalent radius',
                                                                               'VdW radius',
                                                                               'crystal radius',
                                                                               'a x 106 ',
                                                                               'Heat capacity ',
                                                                               'l',
                                                                               'electron affinity ',
                                                                               'VE',
                                                                               'Surface energy '],axis=1)
desc=desc.loc[list(baseatom)+list(proatom),:]
desc = desc.fillna(desc.mean())

In [76]:
# CO oxdation
ntree = 1500
if debug_run:
    ntree = 10

# Lasso
cv_lasso = GridSearchCV(
    Lasso(random_state=929), cv=5,
    param_grid={"alpha": range_lasso})

# Ridge
cv_ridge = GridSearchCV(
    Ridge(random_state=929), cv=5,
    param_grid={"alpha": range_ridge})

# KRR
cv_krr = GridSearchCV(
    KernelRidge(kernel='rbf'), cv=5,
    param_grid={"alpha": range_krr_alpha, 
                "gamma": range_krr_gamma})

# SVR
cv_svr = GridSearchCV(
    SVR(kernel='rbf'), cv=5,
    param_grid={"C": range_svr_C, 
                "gamma": range_svr_gamma, 
                "epsilon": range_svr_epsilon})

# XGB
cv_xgb = GridSearchCV(
    XGBRegressor(random_state=929,
                 n_estimators=ntree), cv=5,
    param_grid={"max_depth": range_depth, 
                "subsample": range_subsample, 
                "colsample_bytree": range_colsample,
                "learning_rate": range_lr})

## Fig.6 (C)

In [77]:
cv_xgb.fit(co_desc.iloc[:,:-1], co_desc.iloc[:,-1])

cvdepth = cv_xgb.best_params_['max_depth']
cvsubsample = cv_xgb.best_params_['subsample']
cvcolsample = cv_xgb.best_params_['colsample_bytree']
cvlr = cv_xgb.best_params_['learning_rate']

print(cvdepth, cvsubsample, cvcolsample, cvlr)

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/vers

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/

3 0.9 1 0.1


In [78]:
estimator = XGBRegressor(n_estimators=ntree,\
                       max_depth=cvdepth,\
                       subsample=cvsubsample,\
                       colsample_bytree=cvcolsample,\
                       learning_rate=cvlr,\
                       random_state=929, seed=929, n_jobs=-1)
print(estimator)

best_xgb_codesc = estimator

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, importance_type='gain',
             learning_rate=0.1, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=None, n_estimators=10, n_jobs=-1,
             nthread=None, objective='reg:linear', random_state=929,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=929,
             silent=True, subsample=0.9)


In [79]:
train_test = shuffle(co_desc, random_state=929)

In [80]:
cv = Cv_Pred_Expected(estimator=estimator,\
                      X=train_test.iloc[:,:-1],\
                      y=train_test.iloc[:,-1],\
                      cv=10, redc=True)
cv.cross_validation()

/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE_test: 36.759 STD_test: 0.905
RMSE_train: 36.605 STD_train: 0.103
Data size:(5567, 115)


In [81]:
xgb_cvbest = cv

In [82]:
cv.plot_pred_expected(save=True,\
                      title='',\
                      os_title='',\
                      filename='output/co_gb_',\
                      titlesize=11,
                      close=True)

(5010, 115) (5010,)


/Users/takigawa/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


RMSE(Train): 36.665
RMSE(Test): 36.831


In [ ]:
plt.figure(figsize=(6, 4), dpi=300)
xgb_cvbest.importance.mean(axis=1).sort_values(ascending=False).iloc[:20][::-1].plot(kind='barh',color='b')
plt.savefig('output/fig6_co.png', format='png',bbox_inches='tight', dpi=300)
plt.savefig('output/fig6_co.pdf', format='pdf',bbox_inches='tight', dpi=300)
plt.close()

## Fig.2 (C)

In [ ]:
common = []
hoge = co.loc[:,'Pt':'Pd']
count = 0
for key, row in hoge.iterrows():
    for i, v in hoge.iloc[count + 1:].iterrows():
        temp = (np.array(row) > 1e-5)
        temp2 = (np.array(v) > 1e-5)
        val = (temp * temp2).sum()
        common.append(val)    
    count = count + 1

freq = dict(Counter(common))
plt.figure(figsize=(1.5, 3))
plt.bar(freq.keys(), freq.values())
plt.xlabel('# Common Elements')
plt.ylabel('Frequency (log)')
plt.yscale('log')
plt.savefig('output/fig2_co.png',format='png', bbox_inches='tight',dpi=300)
plt.savefig('output/fig2_co.pdf',format='pdf', bbox_inches='tight',dpi=300)
plt.close()

## Table 4 (co)

### co_nocond

In [ ]:
co_nocond = pd.concat([co.loc[:,:'Pd'], pro, co.iloc[:,-1]],axis=1)

In [ ]:
rmse_co_nocond = {}
data = shuffle(co_nocond, random_state=10)
target_data = co_nocond

In [ ]:
# Lasso
cv_lasso.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_lasso.best_params_['alpha']
best_lasso = Lasso(random_state=929, alpha=cvalpha)

# Ridge
cv_ridge.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_ridge.best_params_['alpha']
best_ridge = Ridge(random_state=929, alpha=cvalpha)

# KRR
cv_krr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_krr.best_params_['alpha']
cvgamma = cv_krr.best_params_['gamma']
best_krr = KernelRidge(kernel='rbf', alpha=cvalpha, gamma=cvgamma)

# SVR
cv_svr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvC = cv_svr.best_params_['C']
cvgamma = cv_svr.best_params_['gamma']
cvepsilon = cv_svr.best_params_['epsilon']
best_svr = SVR(kernel='rbf', epsilon=cvepsilon, C=cvC, gamma=cvgamma)

# RFR
best_rfr = RandomForestRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# ETR
best_etr = ExtraTreesRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# XGB
cv_xgb.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvdepth = cv_xgb.best_params_['max_depth']
cvsubsample = cv_xgb.best_params_['subsample']
cvcolsample = cv_xgb.best_params_['colsample_bytree']
cvlr = cv_xgb.best_params_['learning_rate']
best_xgb = XGBRegressor(n_estimators=ntree,\
                       max_depth=cvdepth,\
                       subsample=cvsubsample,\
                       colsample_bytree=cvcolsample,\
                       learning_rate=cvlr,\
                       random_state=929, seed=929, n_jobs=-1)

In [ ]:
model = best_lasso
print(model)
rmse_co_nocond['Lasso'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Lasso')

model = best_ridge
print(model)
rmse_co_nocond['Ridge'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10,  'Ridge')

model = best_krr
print(model)
rmse_co_nocond['KRR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'KRR')

model = best_svr
print(model)
rmse_co_nocond['SVR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'SVR')

model = best_rfr
print(model)
rmse_co_nocond['RFR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'RFR')

model = best_etr
print(model)
rmse_co_nocond['ETR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'ETR')

model = best_xgb
print(model)
rmse_co_nocond['XGB'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'XGB')

### co

In [ ]:
rmse_co = {}
data = shuffle(co, random_state=10)
target_data = co

In [ ]:
# Lasso
cv_lasso.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_lasso.best_params_['alpha']
best_lasso = Lasso(random_state=929, alpha=cvalpha)

# Ridge
cv_ridge.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_ridge.best_params_['alpha']
best_ridge = Ridge(random_state=929, alpha=cvalpha)

# KRR
cv_krr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_krr.best_params_['alpha']
cvgamma = cv_krr.best_params_['gamma']
best_krr = KernelRidge(kernel='rbf', alpha=cvalpha, gamma=cvgamma)

# SVR
cv_svr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvC = cv_svr.best_params_['C']
cvgamma = cv_svr.best_params_['gamma']
cvepsilon = cv_svr.best_params_['epsilon']
best_svr = SVR(kernel='rbf', epsilon=cvepsilon, C=cvC, gamma=cvgamma)

# RFR
best_rfr = RandomForestRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# ETR
best_etr = ExtraTreesRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# XGB
cv_xgb.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvdepth = cv_xgb.best_params_['max_depth']
cvsubsample = cv_xgb.best_params_['subsample']
cvcolsample = cv_xgb.best_params_['colsample_bytree']
cvlr = cv_xgb.best_params_['learning_rate']
best_xgb = XGBRegressor(n_estimators=ntree,\
                       max_depth=cvdepth,\
                       subsample=cvsubsample,\
                       colsample_bytree=cvcolsample,\
                       learning_rate=cvlr,\
                       random_state=929, seed=929, n_jobs=-1)

In [ ]:
model = best_lasso
print(model)
rmse_co['Lasso'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Lasso')

model = best_ridge
print(model)
rmse_co['Ridge'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Ridge')

model = best_krr
print(model)
rmse_co['KRR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'KRR')

model = best_svr
print(model)
rmse_co['SVR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'SVR')

model = best_rfr
print(model)
rmse_co['RFR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'RFR')

model = best_etr
print(model)
rmse_co['ETR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'ETR')

model = best_xgb
print(model)
rmse_co['XGB'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'XGB')

### co_desc

In [ ]:
rmse_co_desc = {}
data = shuffle(co_desc, random_state=929)
target_data = co_desc

In [ ]:
# Lasso
cv_lasso.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_lasso.best_params_['alpha']
best_lasso = Lasso(random_state=929, alpha=cvalpha)

# Ridge
cv_ridge.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_ridge.best_params_['alpha']
best_ridge = Ridge(random_state=929, alpha=cvalpha)

# KRR
cv_krr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvalpha = cv_krr.best_params_['alpha']
cvgamma = cv_krr.best_params_['gamma']
best_krr = KernelRidge(kernel='rbf', alpha=cvalpha, gamma=cvgamma)

# SVR
cv_svr.fit(target_data.iloc[:,:-1], target_data.iloc[:,-1])
cvC = cv_svr.best_params_['C']
cvgamma = cv_svr.best_params_['gamma']
cvepsilon = cv_svr.best_params_['epsilon']
best_svr = SVR(kernel='rbf', epsilon=cvepsilon, C=cvC, gamma=cvgamma)

# RFR
best_rfr = RandomForestRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

# ETR
best_etr = ExtraTreesRegressor(n_estimators=ntree, random_state=929, n_jobs=-1)

In [ ]:
model = best_lasso
print(model)
rmse_co_desc['Lasso'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Lasso')

model = best_ridge
print(model)
rmse_co_desc['Ridge'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'Ridge')

model = best_krr
print(model)
rmse_co_desc['KRR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'KRR')

model = best_svr
print(model)
rmse_co_desc['SVR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'SVR')

model = best_rfr
print(model)
rmse_co_desc['RFR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'RFR')

model = best_etr
print(model)
rmse_co_desc['ETR'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'ETR')

model = best_xgb_codesc
print(model)
rmse_co_desc['XGB'] = crossvalid(data.iloc[:,:-1], data.iloc[:,-1], model, 10, 'XGB')

### Table 4

In [ ]:
pd.options.display.precision = 2

print(pd.DataFrame(rmse_co_nocond))

print(pd.DataFrame(rmse_co))

print(pd.DataFrame(rmse_co_desc))